In [1]:
import matplotlib.pyplot as plt
import matplotlib as pmimg
import numpy as np
import skimage
from skimage import color
from skimage import data
from skimage import io
from tqdm import tqdm
import cv2

import os 
dir_path = os.getcwd()

In [2]:
def oneHot(imgTitle):
    animalLabel = imgTitle.split('.')[0][-3:]
    return animalLabel

In [54]:
def createTrainSet():
    trainingSet = []
#     data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/practice-train'
    data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/alternate_data/train'
    for imgLabel in tqdm(os.listdir(data_dir)[:1000]):
        animalLabel = oneHot(imgLabel)
        #dog = [1, 0], cat = [0,1]
        oneHotLabel = []
        if animalLabel == 'dog':
            oneHotLabel = [1, 0]
        elif animalLabel == 'cat':
            oneHotLabel = [0, 1]
        else:
            print('nada')
            continue
        imgPath = os.path.join(data_dir, imgLabel)
        imgObj = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
        imgObj = cv2.resize(imgObj, (56, 56))

        trainingSet.append([imgObj, oneHotLabel])
    return trainingSet

train = createTrainSet()

100%|██████████| 1000/1000 [00:01<00:00, 932.23it/s]


In [55]:
# Filter
# 
side_detector_layer = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])

reverse_side_detector_layer = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

top_bottom_detector_layer = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, 1]
])

angle_detector_layer = np.array([
    [0, 1, 1],
    [-1, 0, 1],
    [-1, -1, 0]
])


top_detector_layer = np.array([
    [1, 1, 1],
    [-1, -1, -1],
    [-1, -1, -1]
])


x_detector_layer = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, 1]
])

right_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1]
]);

left_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0]
]);

small_angle_detector_layer = np.array([
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
])

horiz_filter = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1],
])
vertical_filter = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])
whole_filter = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
])

In [56]:
def relu(imgMatrix):
    filterMask = imgMatrix > 0
    reluMatrix = imgMatrix * filterMask
    return reluMatrix

In [57]:
def convolve_2d(img, filter, padding='same', stride=1, depth=1, dataSetType="test"):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
    padded_img = np.pad(img, ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    
    filterLength = len(filter)

    for zIdx in range(depth):
        for yIdx in range(padded_img.shape[0] - (2 * padding)):
            for xIdx in range(padded_img.shape[1] - (2 * padding)):
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.average(filteredWindow)
                resultArr[zIdx][yIdx][xIdx] = nodeValue
                xIdx += 1
    #Boom!
    return resultArr

    
def createConvolutionalLayer(filtersArr, img):
    convFilterArr = np.zeros((len(filtersArr), img.shape[0], img.shape[1]))
    idx = 0
    for filter in filtersArr:
        convResult = convolve_2d(img, filter, depth=1)
        convFilterArr[idx] = convResult
        idx += 1
    return convFilterArr    

def train_conv_layer(imgSet, filters):
    imgSet = np.array(imgSet)
    x = 0
    print('CONVOLUTIONALLLLLL')
    for img in tqdm(imgSet):
        imgSet[x][0] = createConvolutionalLayer(filters, img[0])
        x += 1
    return imgSet


def relu_layer(imgArr):
    imgObjIdx = 0
    print('RELUUUUUU')
    for imgObj in tqdm(imgArr):
        filterIdx = 0
        listOfFilteredImages = imgObj[0]
        for imgMatrix in listOfFilteredImages:
            reluImg = relu(imgMatrix)
            imgObj[0][filterIdx] = reluImg
            filterIdx += 1
        
        imgArr[imgObjIdx] = imgObj
        imgObjIdx += 1  
    return imgArr


def max_pool_layer(imgArr, filters):
    print(imgArr.shape)
    imgObjIdx = 0
    print('MAX POOOOOLLLLLLL')
    for img in tqdm(imgArr):
        filterIdx = 0
        
        for imgMat in imgArr[imgObjIdx][0]:
            maxedMat = max_pool(imgMat, filters[filterIdx])
            imgArr[imgObjIdx][0][filterIdx] = maxedMat
            filterIdx += 1
            
        imgObjIdx += 1
        
    return imgArr


In [58]:
# Begin training
# trainSet = train
# trainSet = train_conv_layer(trainSet, filters)
# trainSet = relu_layer(trainSet)
# trainSet = max_pool_layer(trainSet, filters)
# print(trainSet[0][0].shape)

In [68]:
# Real image
# Begin single image going through layers
# convolve_2d(img, filter, padding='same', stride=1, depth=1):
# def relu(imgMatrix):
# def max_pool(img, filter, padding='same', stride=1, depth=1):
four_side_detector_layer = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1],
#     [1, 0, -1],
])
filters = [
    four_side_detector_layer,
    side_detector_layer,
    reverse_side_detector_layer,
    small_angle_detector_layer,
    top_bottom_detector_layer,
    angle_detector_layer,
    top_detector_layer,
    left_knight_detector_layer,
    right_knight_detector_layer,
    x_detector_layer
]



def max_pool(img, filter, padding='same', stride=1, depth=1, dataSetType="test"):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = int(len(img[0]) / stride)
    padded_img = np.pad(img[0], ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    filterLength = len(filter)

    for zIdx in range(depth):
        yIdx = 0
        yUpLimit = padded_img.shape[1] - (2 * padding)
        while yIdx + stride - len(filter) < yUpLimit:
            xIdx = 0
            xUpLimit = padded_img.shape[1] - (2 * padding)
            while xIdx + stride - len(filter) < xUpLimit:
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                print(xIdx)
                print(xEnd)
                print('======')
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.amax(filteredWindow)
                resultArr[zIdx][yIdx/stride][xIdx/stride] = nodeValue
                xIdx += stride
            yIdx += stride
    #Boom!
    return resultArr


animalIdx = 999
filter = filters[0]
animalImg = train[animalIdx][0]
answer = False

if (len(animalImg) > 1):
    if train[animalIdx][1][0] == 0 and train[animalIdx][1][1] == 1:
        answer = "cat"
    elif train[animalIdx][1][1] == 0 and train[animalIdx][1][0] == 1:
        answer = "dog"
        
animalImg = convolve_2d(animalImg, filter)
animalImg = relu(animalImg)
animalImg = max_pool(animalImg, filter, stride=1)

animalImg = convolve_2d(animalImg, filter)
animalImg = relu(animalImg)
print(animalImg.shape)
animalImg = max_pool(animalImg, filter, stride=1, depth=len(animalImg))
print(animalImg.shape)

if answer:
    print(answer)
plt.imshow(animalImg[0])

0
3
1
4
2
5
3
6
4
7
5
8
6
9
7
10
8
11
9
12
10
13
11
14
12
15
13
16
14
17
15
18
16
19
17
20
18
21
19
22
20
23
21
24
22
25
23
26
24
27
25
28
26
29
27
30
28
31
29
32
30
33
31
34
32
35
33
36
34
37
35
38
36
39
37
40
38
41
39
42
40
43
41
44
42
45
43
46
44
47
45
48
46
49
47
50
48
51
49
52
50
53
51
54
52
55
53
56
54
57
55
58
56
59


ValueError: could not broadcast input array from shape (2) into shape (3)